In [1]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import pearsonr

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_varspeed_1.npz', allow_pickle=False)

# measurement = np.load('../../dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_reflector_1.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)
bob_channel = 4  # A -> E  (legitimate)

trainData - (6400, 15, 251, 2)
testData - (1600, 15, 251, 2)


In [2]:
def get_features(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    magnitude = np.abs(real + 1j * imag)
    # features = np.hstack((real, imag, magnitude))
    return magnitude

In [3]:
def apply_pca_to_dimension(data, no_components):
    """
    Apply PCA to the 2nd dimension (251 points) of the dataset.
    
    Parameters:
    data (numpy.ndarray): The dataset with shape (n_samples, 251, 2).
    no_components (int): The number of components to reduce to.
    
    Returns:
    numpy.ndarray: The dataset after applying PCA, with shape (n_samples, no_components, 2).
    """
    # Reshape data to apply PCA to the 251 dimension
    # Original shape: (n_samples, 251, 2) --> New shape: (n_samples * 2, 251)
    print(f'data - {data.shape}')
    n_samples = data.shape[0]
    # print(f'n_samples - {n_samples}')
    data_reshaped = data.reshape(-1, 251)
    print(f'data_reshaped - {data_reshaped.shape}')
    print(f'data_reshaped - {data_reshaped[0]}')
    
    # Apply PCA to reduce the 251 dimension to no_components
    pca = PCA(n_components=no_components)
    data_pca = pca.fit_transform(data_reshaped)

    # Reshape data back to original structure with reduced dimension
    # New shape: (n_samples, no_components, 2)
    data_reduced = data_pca.reshape(n_samples, no_components, 2)

    return data_reduced

In [6]:
# Correlation
alice_CIRs = data_cir[:, alice_channel, :, :]
alice_features = get_features(alice_CIRs)

bob_CIRs = data_cir[:, bob_channel, :, :]
bob_features = get_features(bob_CIRs)

eve_CIRs = data_cir[:, eve_channel, :, :]
eve_features = get_features(eve_CIRs)

num_samples = alice_features.shape[0]

correlations_alice_bob = []
correlations_alice_eve = []

for i in range(num_samples):
    # Extract features for the i-th sample
    feature_alice = alice_features[i]
    feature_bob = bob_features[i]
    feature_eve = eve_features[i]
    
    # corr_coeff_ab = np.corrcoef(feature_alice, feature_bob)[0, 1]
    corr_coeff_ab = pearsonr(feature_alice, feature_bob)
    correlations_alice_bob.append(corr_coeff_ab)
    
    # corr_coeff_ae = np.corrcoef(feature_alice, feature_eve)[0, 1]
    corr_coeff_ae = pearsonr(feature_alice, feature_eve)
    correlations_alice_eve.append(corr_coeff_ae)

correlations_alice_bob = np.array(correlations_alice_bob)
correlations_alice_eve = np.array(correlations_alice_eve)

# average
avg_corr_ab = np.mean(correlations_alice_bob)
avg_corr_ae = np.mean(correlations_alice_eve)

print(f"Average Pearson Correlation between Alice and Bob: {avg_corr_ab}")
print(f"Average Pearson Correlation between Alice and Eve: {avg_corr_ae}")


alice_flat = alice_CIRs.flatten()
bob_flat = bob_CIRs.flatten()
eve_flat = eve_CIRs.flatten()


corr_alice_bob_before, _ = pearsonr(alice_flat, bob_flat)
corr_alice_eve_before, _ = pearsonr(alice_flat, eve_flat)
print(f"Before PCA: Pearson Correlation between Alice and Bob: {corr_alice_bob_before}")
print(f"Before PCA: Pearson Correlation between Alice and Eve: {corr_alice_eve_before}")

Average Pearson Correlation between Alice and Bob: 0.40087013446535164
Average Pearson Correlation between Alice and Eve: 0.3861404330776265
Before PCA: Pearson Correlation between Alice and Bob: 0.0009932837302996655
Before PCA: Pearson Correlation between Alice and Eve: 0.001651668108851064


In [4]:


alice_train_CIRs = trainCIR[:, alice_channel, :, :] # (6400, 251, 2)
eve_train_CIRs = trainCIR[:, eve_channel, :, :] # (6400, 251, 2)
train_cirs = np.vstack((alice_train_CIRs, eve_train_CIRs))


n_components = 1
train_cirs_pca = []
pca_attributes = []

# (6400, 251, 2)
for cir in train_cirs:
    scaler = StandardScaler()
    cir_scaled = scaler.fit_transform(cir) # Shape: (251, 2)    
    
    pca = PCA(n_components=1)
    cir_pca_transformed = pca.fit_transform(cir_scaled)  # Shape: (251, n_components)
    
    train_cirs_pca.append(cir_pca_transformed)
    pca_attributes.append({
        'components': pca.components_,
        'explained_variance': pca.explained_variance_
    })

data_reduced = apply_pca_to_dimension(train_cirs, 4)

# train_cirs_pca = np.array(train_cirs_pca)
train_cirs_pca = np.array(data_reduced)
print(f'train_cirs_pca : {train_cirs_pca.shape}')
    
alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(eve_train_CIRs.shape[0])    # Label '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels))

train_atoms = train_cirs_pca.reshape(train_cirs_pca.shape[0], -1)
D = train_atoms.T
print(f'Dictionary shape: {D.shape}')

data - (12800, 251, 2)
data_reshaped - (25600, 251)
data_reshaped - [   -18     34    -42     59    113    132     87    214    198     73
     20     36    -15      4     -3     87     13     73    102     95
     53     98     26    -48     81    -13    169      1    167     69
    181    112     96     28     50     38     80    213    152    249
    134    124    196    130    244     63     90     12     72   -190
    208   -177    348    -26    420    222    312    337     85    268
    -10     79    103    104     98    283    -56    237    -45    145
      4    240    213    -86    333    218    137    362   -279    305
   -123     89     74     72    136     79    171    289    162    454
     58    241    -23    108    -31     77     15     92     59    234
    101   2607   -997  11360  -2819  15324  -3083  10353   2186  -4785
   9230  -9419  13802  -2065  12403   4833   5800   8553  -2127   9586
  -6076   8411  -6056   5238  -3184   1715   -339   1828    358   5871
  -1470  

In [6]:
# Step 3: Extract Features for Test Data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((alice_test_CIRs, eve_test_CIRs))

test_cirs_pca = []
for cir in test_cirs:
    scaler = StandardScaler()
    cir_scaled = scaler.fit_transform(cir) # Shape: (251, 2)    
    
    pca = PCA(n_components=n_components)
    cir_pca_transformed = pca.fit_transform(cir_scaled)  # Shape: (251, n_components)
    
    test_cirs_pca.append(cir_pca_transformed)
    # pca_attributes.append({
    #     'components': pca.components_,
    #     'explained_variance': pca.explained_variance_
    # })

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_CIRs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Combine data and labels for testing
# test_atoms = np.array(test_cirs_pca)
test_atoms = apply_pca_to_dimension(test_cirs, 4)
test_atoms = test_atoms.reshape(test_atoms.shape[0], -1)

print(f'test_atoms : {test_atoms.shape}')

data - (3200, 251, 2)
data_reshaped - (6400, 251)
data_reshaped - [    51    156    -93     39     -6    -78    158     58    -30    190
    -75    167    -95    129     84     38     92    135     79    115
     82    111     60     -1     93     36     13     30     47     24
     55     96     56    136    139    172    252     74    -94    -93
     77    -71    176   -114    146     90   -322    249   -117    137
    241     93    121   -217   -332     41   -137    173   -142    195
     15     59   -198    -77   -138     19     19    109   -166    297
   -222    166    -50     69     -6    217     21    301    110    159
     43     78      8     56     17     90     -2     66    159    215
    113    380    -50    249      3     82     71     61    726   -151
   8475   -745  16408   1125  16275   5258  12199  11395  -4131  13517
 -15908   2324 -12242  -9053    645 -12193   1833  -4368  -3276    516
   1227   -907   9295  -3171   4784  -2216    172    -32   -612    427
   -776   -

In [7]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    # Find the index of the smallest residual
    min_residual_index = np.argmin(residuals)
    # Use this index to find the corresponding class
    predicted_class = unique_classes[min_residual_index]
    
    # print(f'predicted_class : {predicted_class}')
    # print(f'residuals : {residuals}')
    return predicted_class

In [8]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []


for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)


predictions = np.array(predictions)
# print(predictions.shape)

accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

Classification Accuracy: 49.78%


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

In [26]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: (2000,)
tp: 679
tn: 538
fp: 462
fn: 321
MDR: 0.462
FAR: 0.321
AR: 0.6085


In [27]:
alice_flat = alice_train_CIRs.flatten()
eve_flat = eve_train_CIRs.flatten()
bob_flat = bob_train_CIRs.flatten()

# Compute correlation coefficients
corr_alice_bob, _ = pearsonr(alice_flat, bob_flat)
corr_alice_eve, _ = pearsonr(alice_flat, eve_flat)

print(f"Pearson correlation between Alice and Bob features: {corr_alice_bob}")
print(f"Pearson correlation between Alice and Eve features: {corr_alice_eve}")

Pearson correlation between Alice and Bob features: 0.003679370585700648
Pearson correlation between Alice and Eve features: 0.0006604840649587871
